### Build a map, Using GEOPANDAS, to display the price per square meter in each city of belgium

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import geopandas as gpd
import json
from mpl_toolkits.axes_grid1 import make_axes_locatable

### Add column to geojson, price per sqm for each zipcode
geojson file contain geometry related to each city and zip code

In [ ]:
# Create a dictionary "zipcode":price_sqm_meter
zip_pricesqm = pd.read_csv('./zip_price_sqm_meters.csv')
print(zip_pricesqm.head())
zip_pricesqm_dict = zip_pricesqm.set_index('zip_code')['price-sqm'].to_dict()

# Add a property to each featue in the geojson file, named 'mean_price_sqm', according to the zipcode
with open('./datas/georef-belgium-postal-codes.geojson') as f:
    data = json.load(f)

# Iterate over the features
for feature in data['features']:
    # Add the new column to the properties
    _zip = int(feature['properties']['postcode'])
    if _zip in zip_pricesqm_dict:
        print(_zip, zip_pricesqm_dict.get(_zip))
        feature['properties']['mean_price_sqm'] = zip_pricesqm_dict.get(_zip)
    else:
        feature['properties']['mean_price_sqm'] = 0

# Write the modified GeoJSON file to disk
with open('./datas/georef-belgium-postal-codes-mpsqm.geojson', 'w') as f:
    json.dump(data, f)


### Test GeoPandas: display a map from geojson source file

In [ ]:
filename = "./datas/georef-belgium-postal-codes.geojson"
file = open(filename)
fig, ax = plt.subplots(1,1)

belgium_map_df = gpd.read_file(file)
# belgium_map_df.tail()

belgium_map_df.plot(ax=ax, cmap="GnBu", legend=True)
belgium_map_df.set_title = "Belgium"
plt.show()
plt.close()






###

### Display the map with the geojson where psqm was added
issues with the size and position on the graph

In [ ]:

filename = "./datas/georef-belgium-postal-codes-mpsqm.geojson"
file = open(filename)
belgium_map_df = gpd.read_file(file)

fig, ax = plt.subplots(1,figsize=(80,80), facecolor='lightgray')
# color bar
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)
vmax = belgium_map_df['mean_price_sqm'].max()
mappable = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=vmax))
cbar = fig.colorbar(mappable, cax=cax, orientation="horizontal", shrink=0.5)

# Plot Map
belgium_map_df.plot(kind="geo",column='mean_price_sqm',
                    ax=ax,
                    cax=cax,
                    cmap="Reds",
                    edgecolor='black',
                    linewidth=1,
                    )
fig.set_size_inches(50,50)
fig.suptitle("Mean price per zipcode in Belgium\n Houses and Apartments", fontsize=100, color='Black', y=1.2)
cbar.set_label('Price per square meter €/m', fontsize=100)
cbar.set_ticks(np.arange(0, vmax, 1000))
cbar.ax.tick_params(labelsize=50)
# plt.tick_params(left = False, bottom = False)


plt.show()
# ax.set_visible(False)
